IMPORT AND LOAD THE DATA FILE

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random


Using TensorFlow backend.


In [2]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

PREPROCESSING THE DATA

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #Tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #Add documents in the corpus
        documents.append((w, intent['tag']))
        #Add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
#Lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
#Sort classes
classes =  sorted(list(set(classes)))
#Document = combination betweeen patterns and intents
print(len(documents), 'documents')
#Classes = intents
print(len(classes), 'classes', classes)
#Words = all words, vocabulary
print(len(words), 'unique lemmatized words', words)

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


CREATING TRAINING AND TESTING DATA

In [5]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [6]:
#Creating the model - 3 layers.
#First layer - 128 Neurons,
#Second layer - 64 Neurons and
#Third output layer contains number of neurons
#Equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape = (len(train_x[0]),), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = 'softmax'))

#Compile model.
#Stochastic gradient with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr = 0.1, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])

#Fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs = 200, batch_size = 5, verbose = 1)
model.save('chatbot_model.h5', hist)

print('Model Created')

Epoch 1/200
47/47 [==============================] - 0s 9ms/step - loss: 2.2276 - accuracy: 0.1277
Epoch 2/200
47/47 [==============================] - 0s 332us/step - loss: 2.0502 - accuracy: 0.2766
Epoch 3/200
47/47 [==============================] - 0s 166us/step - loss: 1.7792 - accuracy: 0.3830
Epoch 4/200
47/47 [==============================] - 0s 332us/step - loss: 1.8229 - accuracy: 0.5106
Epoch 5/200
47/47 [==============================] - 0s 333us/step - loss: 2.2191 - accuracy: 0.5319
Epoch 6/200
47/47 [==============================] - 0s 332us/step - loss: 5.4794 - accuracy: 0.3404
Epoch 7/200
47/47 [==============================] - 0s 699us/step - loss: 10.5113 - accuracy: 0.2766
Epoch 8/200
47/47 [==============================] - 0s 340us/step - loss: 2764.6487 - accuracy: 0.0851
Epoch 9/200
47/47 [==============================] - 0s 426us/step - loss: 21464167376616.5898 - accuracy: 0.1702
Epoch 10/200
47/47 [==============================] - 0s 161us/step - loss: 

47/47 [==============================] - 0s 332us/step - loss: 2.2337 - accuracy: 0.1489
Epoch 81/200
47/47 [==============================] - 0s 332us/step - loss: 2.2318 - accuracy: 0.1489
Epoch 82/200
47/47 [==============================] - 0s 665us/step - loss: 2.2210 - accuracy: 0.0851
Epoch 83/200
47/47 [==============================] - 0s 471us/step - loss: 2.2357 - accuracy: 0.0851
Epoch 84/200
47/47 [==============================] - 0s 332us/step - loss: 2.2082 - accuracy: 0.1489
Epoch 85/200
47/47 [==============================] - 0s 665us/step - loss: 2.2188 - accuracy: 0.0851
Epoch 86/200
47/47 [==============================] - 0s 332us/step - loss: 2.2277 - accuracy: 0.1489
Epoch 87/200
47/47 [==============================] - 0s 774us/step - loss: 2.2262 - accuracy: 0.1489
Epoch 88/200
47/47 [==============================] - 0s 425us/step - loss: 2.2225 - accuracy: 0.1277
Epoch 89/200
47/47 [==============================] - 0s 434us/step - loss: 2.2281 - accuracy: 

47/47 [==============================] - 0s 332us/step - loss: 2.2222 - accuracy: 0.1489
Epoch 160/200
47/47 [==============================] - 0s 665us/step - loss: 2.2293 - accuracy: 0.1489
Epoch 161/200
47/47 [==============================] - 0s 332us/step - loss: 2.2144 - accuracy: 0.1489
Epoch 162/200
47/47 [==============================] - 0s 333us/step - loss: 2.2325 - accuracy: 0.1489
Epoch 163/200
47/47 [==============================] - 0s 332us/step - loss: 2.2286 - accuracy: 0.0638
Epoch 164/200
47/47 [==============================] - 0s 332us/step - loss: 2.2143 - accuracy: 0.1489
Epoch 165/200
47/47 [==============================] - 0s 332us/step - loss: 2.2254 - accuracy: 0.1489
Epoch 166/200
47/47 [==============================] - 0s 332us/step - loss: 2.2144 - accuracy: 0.1064
Epoch 167/200
47/47 [==============================] - 0s 333us/step - loss: 2.2263 - accuracy: 0.1489
Epoch 168/200
47/47 [==============================] - 0s 332us/step - loss: 2.2217 - a